In [31]:
import json
from collections import defaultdict,Counter

path='E://study//Demo//Data_Demo//Python//resource//datasets//example.txt'
records = [json.loads(line) for line in open(path)]
time_zones = [rec['tz'] for rec in records if 'tz' in rec]
time_zones[:3]

['America/New_York', 'America/Denver', 'America/New_York']

In [32]:
def get_totals(sequence):
    counts={}
    for x in sequence:
        if x in counts:
            counts[x]+=1
        else:
            counts[x]=1
    return counts

def get_totals_2(sequence):
    counts=defaultdict(int)
    for x in sequence:
        counts[x]+=1
    return counts

def top_counts(count_dict,n=10):
    value_key_pairs = [(count, tz) for tz, count in count_dict.items()]
    value_key_pairs.sort()
    return value_key_pairs[-n:]

counts = get_totals(time_zones)
top_counts(counts)        

counts_2 = Counter(time_zones)
counts_2.most_common(10)

[('America/New_York', 1251),
 ('', 521),
 ('America/Chicago', 400),
 ('America/Los_Angeles', 382),
 ('America/Denver', 191),
 ('Europe/London', 74),
 ('Asia/Tokyo', 37),
 ('Pacific/Honolulu', 36),
 ('Europe/Madrid', 35),
 ('America/Sao_Paulo', 33)]